_Lambda School Data Science — Big Data_

# AWS SageMaker

### Links

#### AWS
- The Open Guide to Amazon Web Services: EC2 Basics _(just this one short section!)_ https://github.com/open-guides/og-aws#ec2-basics
- AWS in Plain English https://www.expeditedssl.com/aws-in-plain-english
- Amazon SageMaker » Create an Amazon SageMaker Notebook Instance https://docs.aws.amazon.com/sagemaker/latest/dg/gs-setup-working-env.html
- Amazon SageMaker » Install External Libraries https://docs.aws.amazon.com/sagemaker/latest/dg/nbi-add-external.html

#### Dask
- Why Dask? https://docs.dask.org/en/latest/why.html
- Use Cases https://docs.dask.org/en/latest/use-cases.html
- User Interfaces https://docs.dask.org/en/latest/user-interfaces.html

#### Numba
- A ~5 minute guide http://numba.pydata.org/numba-doc/latest/user/5minguide.html

## 1. Estimate pi
https://en.wikipedia.org/wiki/Approximations_of_π#Summing_a_circle's_area

### With plain Python

In [1]:
import random

def monte_carlo_pi(nsamples):
    acc = 0
    for _ in range(int(nsamples)):
        x = random.random()
        y = random.random()
        if (x**2 + y**2) < 1.0:
            acc += 1
    return 4.0 * acc / nsamples

In [2]:
%%time
monte_carlo_pi(1e7)

CPU times: user 4.22 s, sys: 0 ns, total: 4.22 s
Wall time: 4.22 s


3.1421132

### With Numba
http://numba.pydata.org/

In [6]:
from numba import njit

In [7]:
@njit


def monte_carlo_pi(nsamples):
    acc = 0
    for _ in range(int(nsamples)):
        x = random.random()
        y = random.random()
        if (x**2 + y**2) < 1.0:
            acc += 1
    return 4.0 * acc / nsamples

In [10]:
%%time
monte_carlo_pi(1e7)

CPU times: user 132 ms, sys: 3.82 ms, total: 136 ms
Wall time: 136 ms


3.141416

## 2. Loop a slow function

### With plain Python

In [11]:
from time import sleep

def slow_square(x):
    sleep(1)
    return x**2

In [12]:
%%time
[slow_square(n) for n in range(16)]

# example of a workload you can parallelize

CPU times: user 1.52 ms, sys: 382 µs, total: 1.9 ms
Wall time: 16 s


[0, 1, 4, 9, 16, 25, 36, 49, 64, 81, 100, 121, 144, 169, 196, 225]

### With Dask
- https://examples.dask.org/delayed.html
- http://docs.dask.org/en/latest/setup/single-distributed.html

In [13]:
from dask import compute, delayed

In [15]:
[delayed(slow_square)(n) for n in range(16)]

[Delayed('slow_square-7f50acb9-e1e5-4253-bd3e-d2abaf38dad8'),
 Delayed('slow_square-2800a7ed-2002-42ae-8f18-88f97f156177'),
 Delayed('slow_square-b3c7f105-cc91-4d67-8b21-46e6902b8837'),
 Delayed('slow_square-b8cc10b7-e4c9-427b-b0eb-ebcf821e1ca0'),
 Delayed('slow_square-ec6fa538-e2bf-46f0-9230-8c0a6a362deb'),
 Delayed('slow_square-7413d2e5-2f02-40e5-bc04-2ec31ac67c5b'),
 Delayed('slow_square-cb5204e8-d843-4887-955d-0911837143bc'),
 Delayed('slow_square-b1e3fd29-2eb5-452f-99ef-8a2758dcb19f'),
 Delayed('slow_square-00552788-c393-47d2-8499-1c1935499329'),
 Delayed('slow_square-cbac7dad-60ac-4a44-a969-7ec8acf03433'),
 Delayed('slow_square-c4579af4-72c1-46e0-bb94-1b6c86fc7804'),
 Delayed('slow_square-eb1a9efd-49f9-4974-a8f1-944c7d6a3d68'),
 Delayed('slow_square-39bfbc1f-5d7d-44a6-8a1b-09bfe7387709'),
 Delayed('slow_square-06321bd9-afdb-41a3-a83f-d4ba0e19f82d'),
 Delayed('slow_square-1024072d-9a5b-437d-9a65-46d9f19632e0'),
 Delayed('slow_square-0871e5be-bdc0-433a-a568-97a5808e4b89')]

In [17]:
%%time
compute(delayed(slow_square)(n) for n in range(16))

CPU times: user 7.6 ms, sys: 4.95 ms, total: 12.6 ms
Wall time: 1.01 s


([0, 1, 4, 9, 16, 25, 36, 49, 64, 81, 100, 121, 144, 169, 196, 225],)

## 3. Analyze millions of Instacart orders

### Download data
https://tech.instacart.com/3-million-instacart-orders-open-sourced-d40d29ead6f2

In [18]:
!wget https://s3.amazonaws.com/instacart-datasets/instacart_online_grocery_shopping_2017_05_01.tar.gz

--2019-02-25 18:03:16--  https://s3.amazonaws.com/instacart-datasets/instacart_online_grocery_shopping_2017_05_01.tar.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.9.221
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.9.221|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 205548478 (196M) [application/x-gzip]
Saving to: ‘instacart_online_grocery_shopping_2017_05_01.tar.gz’

instacart_online_gr 100%[===================>] 196.03M   100MB/s    in 2.0s    

2019-02-25 18:03:18 (100 MB/s) - ‘instacart_online_grocery_shopping_2017_05_01.tar.gz’ saved [205548478/205548478]



In [19]:
!tar --gunzip --extract --verbose --file=instacart_online_grocery_shopping_2017_05_01.tar.gz

instacart_2017_05_01/
instacart_2017_05_01/._aisles.csv
instacart_2017_05_01/aisles.csv
instacart_2017_05_01/._departments.csv
instacart_2017_05_01/departments.csv
instacart_2017_05_01/._order_products__prior.csv
instacart_2017_05_01/order_products__prior.csv
instacart_2017_05_01/._order_products__train.csv
instacart_2017_05_01/order_products__train.csv
instacart_2017_05_01/._orders.csv
instacart_2017_05_01/orders.csv
instacart_2017_05_01/._products.csv
instacart_2017_05_01/products.csv


In [20]:
%cd instacart_2017_05_01

/home/ec2-user/SageMaker/DS-Unit-3-Sprint-3-Big-Data/module1-aws-sagemaker/instacart_2017_05_01


In [21]:
!ls -lh *.csv

-rw-r--r-- 1 ec2-user ec2-user 2.6K May  2  2017 aisles.csv
-rw-r--r-- 1 ec2-user ec2-user  270 May  2  2017 departments.csv
-rw-r--r-- 1 ec2-user ec2-user 551M May  2  2017 order_products__prior.csv
-rw-r--r-- 1 ec2-user ec2-user  24M May  2  2017 order_products__train.csv
-rw-r--r-- 1 ec2-user ec2-user 104M May  2  2017 orders.csv
-rw-r--r-- 1 ec2-user ec2-user 2.1M May  2  2017 products.csv


### With Pandas

#### Load & merge data

In [22]:
import pandas as pd

In [23]:
%%time
order_products = pd.concat([
    pd.read_csv('order_products__prior.csv'), 
    pd.read_csv('order_products__train.csv')])

order_products.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 33819106 entries, 0 to 1384616
Data columns (total 4 columns):
order_id             int64
product_id           int64
add_to_cart_order    int64
reordered            int64
dtypes: int64(4)
memory usage: 1.3 GB
CPU times: user 10.4 s, sys: 2.09 s, total: 12.5 s
Wall time: 12.5 s


In [24]:
order_products.head()

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


In [25]:
products = pd.read_csv('products.csv')
products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49688 entries, 0 to 49687
Data columns (total 4 columns):
product_id       49688 non-null int64
product_name     49688 non-null object
aisle_id         49688 non-null int64
department_id    49688 non-null int64
dtypes: int64(3), object(1)
memory usage: 1.5+ MB


In [26]:
products.head()

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


In [27]:
%%time
order_products = pd.merge(order_products, products[['product_id', 'product_name']])

CPU times: user 7.7 s, sys: 2.09 s, total: 9.79 s
Wall time: 9.78 s


In [28]:
order_products.head()

,order_id,product_id,add_to_cart_order,reordered,product_name
0,2,33120,1,1,Organic Egg Whites
1,26,33120,5,0,Organic Egg Whites
2,120,33120,13,0,Organic Egg Whites
3,327,33120,5,1,Organic Egg Whites
4,390,33120,28,1,Organic Egg Whites


#### Most popular products?

In [29]:
order_products.describe()

,order_id,product_id,add_to_cart_order,reordered
count,3.381911e+07,3.381911e+07,3.381911e+07,3.381911e+07
mean,1.710566e+06,2.557551e+04,8.367738e+00,5.900617e-01
std,9.874008e+05,1.409770e+04,7.139540e+00,4.918220e-01
min,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00
25%,8.554130e+05,1.351900e+04,3.000000e+00,0.000000e+00
50%,1.710660e+06,2.525600e+04,6.000000e+00,1.000000e+00
75%,2.565587e+06,3.793500e+04,1.100000e+01,1.000000e+00
max,3.421083e+06,4.968800e+04,1.450000e+02,1.000000e+00


In [36]:
order_products['product_name'].value_counts()


Banana                                                                     491291
Bag of Organic Bananas                                                     394930
Organic Strawberries                                                       275577
Organic Baby Spinach                                                       251705
Organic Hass Avocado                                                       220877
Organic Avocado                                                            184224
Large Lemon                                                                160792
Strawberries                                                               149445
Limes                                                                      146660
Organic Whole Milk                                                         142813
Organic Raspberries                                                        142603
Organic Yellow Onion                                                       117716
Organic Garlic  

#### Organic?

In [37]:
order_products['product_name'].unique()

array(['Organic Egg Whites', 'Michigan Organic Kale', 'Garlic Powder',
       ..., 'Ultra Sun Blossom Liquid 90 loads Fabric Enhancers',
       'Sweetart Jelly Beans', 'Water With Electrolytes'], dtype=object)

In [38]:
order_products['organic'] = order_products['product_name'].str.contains('Organic')

In [39]:
order_products['organic'].value_counts()

False    23163118
True     10655988
Name: organic, dtype: int64

### With Dask
https://examples.dask.org/dataframe.html

In [40]:
import dask.dataframe as dd
from dask.distributed import Client

#### Load & merge data
https://examples.dask.org/dataframes/01-data-access.html#Read-CSV-files

In [41]:
client = Client(n_workers=16)
client

Client Scheduler: tcp://127.0.0.1:38165 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 16 Cores: 16 Memory: 67.53 GB


In [42]:
%%time
order_products = dd.read_csv('order_products*.csv')

CPU times: user 18.7 ms, sys: 3.72 ms, total: 22.4 ms
Wall time: 20.4 ms


http://docs.dask.org/en/latest/dataframe-performance.html#persist-intelligently

In [43]:
order_products

,order_id,product_id,add_to_cart_order,reordered
npartitions=11,,,,
,int64,int64,int64,int64
,...,...,...,...
...,...,...,...,...
,...,...,...,...
,...,...,...,...


#### Most popular products?

In [44]:
order_products = dd.merge(order_products, products[['product_id', 'product_name']])

In [46]:
order_products = order_products.persist()

In [49]:
%%time
order_products['product_name'].value_counts().compute()

CPU times: user 240 ms, sys: 41.5 ms, total: 281 ms
Wall time: 1.53 s


Banana                                                       491291
Bag of Organic Bananas                                       394930
Organic Strawberries                                         275577
Organic Baby Spinach                                         251705
Organic Hass Avocado                                         220877
Organic Avocado                                              184224
Large Lemon                                                  160792
Strawberries                                                 149445
Limes                                                        146660
Organic Whole Milk                                           142813
Organic Raspberries                                          142603
Organic Yellow Onion                                         117716
Organic Garlic                                               113936
Organic Zucchini                                             109412
Organic Blueberries                             

#### Organic?

In [51]:
%%time
order_products['organic'] = order_products['product_name'].str.contains('Organic')
order_products['organic'].value_counts().compute()

CPU times: user 2.05 s, sys: 386 ms, total: 2.44 s
Wall time: 16.3 s


## 4. Fit a machine learning model

### Load data

In [52]:
%cd ../ds1-predictive-modeling-challenge

/home/ec2-user/SageMaker/DS-Unit-3-Sprint-3-Big-Data/module1-aws-sagemaker/ds1-predictive-modeling-challenge


In [53]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier

train_features = pd.read_csv('train_features.csv')
train_labels = pd.read_csv('train_labels.csv')

X_train_numeric = train_features.select_dtypes(np.number)
y_train = train_labels['status_group']

### With 2 cores (like Google Colab)
https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html

In [54]:
model = RandomForestClassifier(n_estimators=200, oob_score=True, n_jobs=2, random_state=42, verbose=1)
model.fit(X_train_numeric, y_train)
print('Out-of-bag score:', model.oob_score_)

[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    2.8s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:   11.7s
[Parallel(n_jobs=2)]: Done 200 out of 200 | elapsed:   12.0s finished


Out-of-bag score: 0.7206397306397306


### With 16 cores (on AWS m4.4xlarge)

In [55]:
model = RandomForestClassifier(n_estimators=200, oob_score=True, n_jobs=16, random_state=42, verbose=1)
model.fit(X_train_numeric, y_train)
print('Out-of-bag score:', model.oob_score_)

[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.4s
[Parallel(n_jobs=16)]: Done 168 tasks      | elapsed:    2.2s
[Parallel(n_jobs=16)]: Done 200 out of 200 | elapsed:    2.5s finished


Out-of-bag score: 0.7206397306397306


## ASSIGNMENT

Revisit a previous assignment or project that had slow speeds or big data.

Make it better with what you've learned today!

You can use `wget` or Kaggle API to get data. Some possibilities include:

- https://www.kaggle.com/c/ds1-predictive-modeling-challenge
- https://www.kaggle.com/ntnu-testimon/paysim1
- https://github.com/mdeff/fma
- https://tech.instacart.com/3-million-instacart-orders-open-sourced-d40d29ead6f2 



Also, you can play with [Datashader](http://datashader.org/) and its [example datasets](https://github.com/pyviz/datashader/blob/master/examples/datasets.yml)!